In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wget
from os.path import exists
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
from flask_ngrok import run_with_ngrok
from flask import Flask, request
import requests

In [7]:
if not exists('data/auto-mpg.csv'):
    wget.download('https://raw.githubusercontent.com/vsvale/datasets/main/auto-mpg.csv','data')

In [8]:
automobile_df = pd.read_csv('data/auto-mpg.csv')
automobile_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [9]:
automobile_df.shape

(398, 9)

In [10]:
automobile_df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weight            int64
acceleration    float64
model year        int64
origin            int64
car name         object
dtype: object

In [11]:
automobile_df['horsepower'] = pd.to_numeric(automobile_df['horsepower'],errors='coerce')

In [12]:
automobile_df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [13]:
automobile_df=automobile_df.fillna(automobile_df.mean())

/tmp/ipykernel_197/3983508211.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  automobile_df=automobile_df.fillna(automobile_df.mean())


In [14]:
automobile_df.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [15]:
automobile_df=automobile_df.drop(['car name'],axis=1)

In [16]:
automobile_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1


In [17]:
X = automobile_df.drop('mpg',axis=1)
Y = automobile_df['mpg']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2)
linear_model = LinearRegression(normalize=True).fit(x_train,y_train)
print('R2: ',linear_model.score(x_train,y_train))
y_pred = linear_model.predict(x_test)
print('Mean Squared Error: ',mean_squared_error(y_test,y_pred))

R2:  0.814969890911259
Mean Squared Error:  8.160462258219887


In [20]:
print(linear_model.predict([[6,117,100,3000,15,85,1]]))

[27.20759256]


In [22]:
pickle.dump(linear_model,open('model.pkl','wb'))

In [23]:
model = pickle.load(open('model.pkl','rb'))

In [25]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/predict',methods=['POST'])
def home():
  cylinders = int(request.args.get('cylinders',''))
  displacement = int(request.args.get('displacement',''))
  horsepower = int(request.args.get('horsepower',''))
  weight = int(request.args.get('weight',''))
  acceleration = int(request.args.get('acceleration',''))
  modelYear = int(request.args.get('modelYear',''))
  origin = int(request.args.get('origin',''))
  prediction = model.predict([[cylinders,displacement,horsepower,weight,acceleration,modelYear,origin]])
  #prediction = model.predict([[6,117,100,3000,15,85,1]])
  print('******************************************')
  print(prediction)
  return 'prediction is '+ str(prediction[0])

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
  File "/home/dataeng/.local/lib/python3.10/site-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/home/dataeng/.local/lib/python3.10/site-packages/flask_ngrok.py", line 31, in _run_ngrok
    ngrok = subprocess.Popen([executable, 'http', '5000'])
  File "/usr/lib/python3.10/subprocess.py", line 969, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr/lib/python3.10/subprocess.py", line 1845, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename)
PermissionError: [Errno 13] Permission denied: '/tmp/ngrok/ngrok'
